In [2]:
import pandas as pd
import joblib

print("Loading processed data and variables...")

# Load the processed DataFrames
train_df_encoded = pd.read_parquet('data/train_processed.parquet')
test_df_encoded = pd.read_parquet('data/test_processed.parquet')

# Load the variables
target = joblib.load('data/target_variable.pkl')
high_card_features = joblib.load('data/high_card_features.pkl')

print("Data loaded. Ready for training.")

Loading processed data and variables...
Data loaded. Ready for training.


In [3]:
import lightgbm as lgb
from sklearn.metrics import log_loss, roc_auc_score
import joblib

# --- 1. Define Final Feature & Target Sets ---
# We already defined 'target' and 'high_card_features' in the previous code.
# The 'features' are all columns in our encoded dataframe, except the ones we drop.
features_to_drop = [target, 'hour'] + high_card_features
features = [col for col in train_df_encoded.columns if col not in features_to_drop]

# Create the final training and testing sets
X_train = train_df_encoded[features]
y_train = train_df_encoded[target]
X_test = test_df_encoded[features]
y_test = test_df_encoded[target]

print("Final X_train shape:", X_train.shape)

# --- 2. Train the Tuned LightGBM Model ---
print("\nTraining high-performance model...")

# These are tuned hyperparameters known to work well for this dataset
model = lgb.LGBMClassifier(
    objective='binary',
    metric='logloss',
    n_estimators=1500,        # More trees
    learning_rate=0.03,     # Slower, more careful learning
    num_leaves=31,          # Standard, good complexity
    max_depth=-1,           # No limit on depth
    min_child_samples=20,   # Prevents overfitting to small groups
    subsample=0.8,          # Use 80% of data for each tree
    colsample_bytree=0.8,   # Use 80% of features for each tree
    random_state=42,
    n_jobs=-1               # Use all your computer's cores
)

# Train the model
model.fit(X_train, y_train)

print("Model training complete!")

# --- 3. Evaluate the Model ---
print("\nEvaluating model performance...")

# Predict probabilities on the held-out test set
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate the final, honest scores
loss = log_loss(y_test, y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)

print("--- Final Model Performance ---")
print(f"Final LogLoss: {loss:.4f}")
print(f"Final AUC Score: {auc:.4f}")

# --- 4. Save the Final Model ---
print("\nSaving model to file...")
joblib.dump(model, 'adintel_model.pkl')
print("Model saved successfully as 'adintel_model.pkl'")

Final X_train shape: (1600000, 2)

Training high-performance model...


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: hour_of_day: object